# merge virus and host result

In [1]:
import pandas as pd
import numpy as np

host_dict = {}
hic = open('/media/chengzw_bk/revision/host_predict/v_2_h_HIC.ntw','r')
for line in hic:
    line = line.strip()
    if ('virus' in line):
        #print('NOV')
        continue
    else:
        line = line.split()
        host_dict[line[0],line[1]] = [1,0,0]
hic.close()

crisp = open('/media/chengzw_bk/revision/host_predict/v_2_h_CRISPR.ntw','r')
for line in crisp:
    line = line.strip()
    if ('virus' in line):
        continue
    else:
        line = line.split()
        if (line[0],line[1]) in host_dict.keys():
            #print('CRISP-HIC',line[0],line[1])
            host_dict[line[0],line[1]][1] = 1
        else:
            host_dict[line[0],line[1]] = [0,1,0]
crisp.close()

blastn = open('/media/chengzw_bk/revision/host_predict/v_2_h_BLASTN.ntw','r')
for line in blastn:
    line = line.strip()
    if ('virus' in line):
        continue
    else:
        line = line.split()
        if (line[0],line[1]) in host_dict.keys():
            host_dict[line[0],line[1]][2] = 1
            #print('blastn',line[0],line[1],host_dict[line[0],line[1]])
            #print(host_dict[line[0],line[1]])
        else:
            host_dict[line[0],line[1]] = [0,0,1]
blastn.close()

v_2_h_merge = open("./v_2_h_merge.txt", 'w')
v_2_h_merge.write('viruse\t'+'host\t'+'hic\t'+'crisp\t'+'blastn\n')
for keys,values in host_dict.items():
    v_2_h_merge.write(keys[0]+'\t'+keys[1]+'\t'+str(values[0])+'\t'+str(values[1])+'\t'+str(values[2])+'\n')
v_2_h_merge.close()

####存在的问题：hic和crispr没有预测到同一个宿主关系，但是hic和blastn有26个共同的预测的宿主

# add host tax and cluster

In [2]:
import pandas as pd
import numpy as np

virus2host = pd.read_table("/media/chengzw_bk/revision/host_predict/v_2_h_merge.txt")
tax = pd.read_table("/media/chengzw_bk/ws1_disk4_bk/gw_assemble_contig/ORFpipe2/all_assemble_1kb+.fasta.1kb+_taxa.tab")
name = pd.read_table("/media/chengzw_bk/ws1_disk4_bk/gw_assemble_contig/ORFpipe2/all_assemble_1kb+.fasta.1kb+_original.name", names=["host","contig"])

tax_name =  pd.merge(name,tax,how='left',on=['contig'])
virus2host_tax = pd.merge(virus2host,tax_name,how='left',on = ['host'])

In [4]:
import pandas as pd
import numpy as np

#virus2host_tax = pd.read_table("/home/disk4-8T/chengzhanwen/host_predict/merge/virus2host_tax.txt")
gbg = pd.read_csv("/media/chengzw_bk/revision/vcontact2/genome_by_genome_overview_exclude5kb.csv")
#ntw = pd.read_csv("/home/disk4-8T/chengzhanwen/onefour_meta_10g/v+m_vcontact2/vcontact_result2/c1.clusters")

# for v in virus2host_tax.viruse:
#     if 'Overlap' in (gbg.loc[gbg['Genome']== v,'VC Status'].values[0]):
#         vc = gbg.loc[gbg['Genome']== v,'VC Status'].values[0].split()[1][1:-1]
#         virus2host_tax.loc[virus2host_tax['viruse']== v,'VC'] = vc
#         virus2host_tax.loc[virus2host_tax['viruse']== v,'VC_Status'] = 'Overlap'
#     elif (gbg.loc[gbg['Genome']== v,'VC Status'].values[0]) in ['Clustered','Clustered/Singleton']:
#         virus2host_tax.loc[virus2host_tax['viruse']== v,'VC'] = gbg.loc[gbg['Genome']== v,'VC Subcluster'].values[0]
#         virus2host_tax.loc[virus2host_tax['viruse']== v,'VC_Status'] = (gbg.loc[gbg['Genome']== v,'VC Status'].values[0])
#     else:
#         virus2host_tax.loc[virus2host_tax['viruse']== v,'VC_Status'] = (gbg.loc[gbg['Genome']== v,'VC Status'].values[0])

####只保留Clustered和singleton的状态
for v in virus2host_tax.viruse:
    if (gbg.loc[gbg['Genome']== v,'VC Status'].values[0]) in ['Clustered','Clustered/Singleton']:
        virus2host_tax.loc[virus2host_tax['viruse']== v,'VC'] = gbg.loc[gbg['Genome']== v,'VC Subcluster'].values[0]
        virus2host_tax.loc[virus2host_tax['viruse']== v,'VC_Status'] = (gbg.loc[gbg['Genome']== v,'VC Status'].values[0])
    else:
        virus2host_tax.loc[virus2host_tax['viruse']== v,'VC_Status'] = (gbg.loc[gbg['Genome']== v,'VC Status'].values[0])

virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','species'] = np.nan
virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','genus'] = np.nan
virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','family'] = np.nan
virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','order'] = np.nan
virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','class'] = np.nan
virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','phylum'] = np.nan
virus2host_tax.loc[virus2host_tax['kingdom'] == 'Viruses','kingdom'] = np.nan
        
virus2host_tax.to_csv("./v_2_h_merge_tax.txt",sep='\t',index=False)

In [5]:
import pandas as pd
from numpy import nan

viral_cluster = pd.read_csv(
    "/media/chengzw_bk/revision/vcontact2/viral_cluster_overview_27taxedVC.csv", sep=',')
virus2host_tax = pd.read_csv(
    "/media/chengzw_bk/revision/host_predict/v_2_h_merge_tax.txt", sep='\t')
viral_cluster = viral_cluster[['VC','Class','Family','Genus']]
viral_cluster.columns = ['VC','V_Class','V_Family','V_Genus']
viral_cluster = viral_cluster.fillna('Unassigned')
#26*4

# virus2host_tax.loc[:,'V_Class'] = 'Unassigned'
# virus2host_tax.loc[:,'V_Family'] = 'Unassigned'
# virus2host_tax.loc[:,'V_Genus'] = 'Unassigned'
#8201*18

virus2host_tax = pd.merge(virus2host_tax,viral_cluster,how='left')
# for vc in viral_cluster.VC:####vc存在nan
#     #print(vc)
#     if vc in virus2host_tax.VC:
#         print(VC)
#         virus2host_tax.loc[virus2host_tax['VC'] == vc,
#                            'V_Class'] = viral_cluster.loc[viral_cluster['VC'] == vc, 'V_Class'].values[0]
#         virus2host_tax.loc[virus2host_tax['VC'] == vc,
#                            'V_Family'] = viral_cluster.loc[viral_cluster['VC'] == vc, 'V_Family'].values[0]
#         virus2host_tax.loc[virus2host_tax['VC'] == vc,
#                            'V_Genus'] = viral_cluster.loc[viral_cluster['VC'] == vc, 'V_Genus'].values[0]
#     else:
#         continue

#virus2host_tax
virus2host_tax.to_csv("./v_2_h_merge_tax_final.txt",sep='\t',index=False)

In [9]:
import pandas as pd
virus2host_tax = pd.read_table("./v_2_h_merge_tax_final.txt")
virus2host_tax['host'].str.split('_',expand=True)[[5]]

,5
0,5119
1,1616
2,4900
3,17402
4,1013
...,...
8196,491687
8197,199627
8198,9940
8199,16379


# add VC coverage and host coverage

In [6]:
import pandas as pd
import numpy as np

virus2host_tax = pd.read_table("./v_2_h_merge_tax_final.txt")
virus2host_tax = virus2host_tax[['class','VC']]
#virus2host_tax = virus2host_tax.rename({'class':'Class'})
####存在overlap对应一个order以及多个vc对应一个order
####overlap的情况将vc1/vc2 拆分，分别对应order
####同一个vc对应order对了多次，还是算一次并去重复
virus2host_tax = virus2host_tax[(~(pd.isnull(virus2host_tax['class']))) & (~(pd.isnull(virus2host_tax['VC'])))]
#virus2host_tax["vc"] = virus2host_tax["VC"].map(lambda x:x.split("/"))
#virus2host_tax = virus2host_tax.explode("vc")[['order','vc']].drop_duplicates()
vc_coverage = pd.read_table("/media/chengzw_bk/revision/coverage/cluster_cov.txt")
vc_coverage['ave'] = vc_coverage.iloc[:,1:5].mean(axis=1)
####存在VC_128_1,VC_175_1,VC_179_1,_2,_3,VC_404_2,VC_431_1,VC_588_1,VC_652_1,VC_766_1不是以VC_*_0结尾
class_coverage = pd.read_table("/media/chengzw_bk/ws1_disk4_bk/four_meta_20g/spades/class_cov.txt")
class_coverage['ave'] = class_coverage.iloc[:, 5:9].mean(axis=1)

for i in virus2host_tax['class']:
    if pd.isnull(i):
        continue
    else:
        #print(i)
        #print(order_coverage.loc[order_coverage['order'] == i,'ave'].values[0])
        virus2host_tax.loc[virus2host_tax['class'] == i,'class_cov'] = class_coverage.loc[class_coverage['class'] == i,'ave'].values[0]
####存在一些vc没有coverage信息       
for v in virus2host_tax.VC:
    if v in vc_coverage.VC.unique():
        virus2host_tax.loc[virus2host_tax['VC'] == v,'vc_cov'] = vc_coverage.loc[vc_coverage['VC'] == v,'ave'].values[0]
#     elif v+'_0' in vc_coverage.VC.unique():
#         #print(v)
#         virus2host_tax.loc[virus2host_tax['vc'] == v,'vc_cov'] = vc_coverage.loc[vc_coverage['VC'] == v+'_0','ave'].values[0]
    else:
        #print(v)
        virus2host_tax.loc[virus2host_tax['VC'] == v,'vc_cov'] = 0
virus2host_tax = virus2host_tax.drop_duplicates()
virus2host_tax_group = virus2host_tax.groupby(['class']).agg({'class_cov':'mean','vc_cov':'sum'}).reset_index()
virus2host_tax_group.to_csv("./class_cov2vc_cov.txt",sep='\t',index=False)

# add viral coverage and host coverage into the table separete prediction

In [9]:
import pandas as pd
import numpy as np

virus2host_tax = pd.read_table("/media/chengzw_bk/revision/host_predict/v_2_h_merge_tax_final.txt")
#virus2host_tax = virus2host_tax[virus2host_tax.blastn == 1].reset_index()[['order','VC']]
virus2host_tax = virus2host_tax[virus2host_tax.blastn == 1].reset_index()[['class','VC']]
#virus2host_tax = virus2host_tax[(virus2host_tax.blastn == 1)|(virus2host_tax.crisp == 1)].reset_index()[['order','VC']]
virus2host_tax = virus2host_tax[(~pd.isnull(virus2host_tax['class'])) & (~(pd.isnull(virus2host_tax['VC'])))]
#virus2host_tax["vc"] = virus2host_tax["VC"].map(lambda x:x.split("/"))
virus2host_tax = virus2host_tax.explode("VC")[['class','VC']].drop_duplicates().reset_index(drop = True)
vc_coverage = pd.read_table("/media/chengzw_bk/revision/coverage/cluster_cov.txt")
vc_coverage['ave'] = vc_coverage.iloc[:, 1:5].mean(axis=1)
####存在VC_128_1,VC_175_1,VC_179_1,_2,_3,VC_404_2,VC_431_1,VC_588_1,VC_652_1,VC_766_1不是以VC_*_0结尾
class_coverage = pd.read_table("/media/chengzw_bk/ws1_disk4_bk/four_meta_20g/spades/class_cov.txt")
class_coverage['ave'] = class_coverage.iloc[:, 5:9].mean(axis=1)
for i in virus2host_tax['class']:
    if pd.isnull(i):
        continue
    else:
        #print(i)
        #print(order_coverage.loc[order_coverage['order'] == i,'ave'].values[0])
        virus2host_tax.loc[virus2host_tax['class'] == i,'class_cov'] = class_coverage.loc[class_coverage['class'] == i,'ave'].values[0]
####存在一些vc没有coverage信息       
for v in virus2host_tax['VC']:
    if v in vc_coverage.VC.unique():
        virus2host_tax.loc[virus2host_tax['VC'] == v,'vc_cov'] = vc_coverage.loc[vc_coverage['VC'] == v,'ave'].values[0]
#     elif v+'_0' in vc_coverage.VC.unique():
#         virus2host_tax.loc[virus2host_tax['vc'] == v,'vc_cov'] = vc_coverage.loc[vc_coverage['VC'] == v+'_0','ave'].values[0]
    else:
        #print(v)
        virus2host_tax.loc[virus2host_tax['VC'] == v,'vc_cov'] = 0        
virus2host_tax_group = virus2host_tax.groupby(['class']).agg({'class_cov':'mean','vc_cov':'sum'}).reset_index()
virus2host_tax_group.to_csv("./class_cov2vc_cov_blastn.txt",sep='\t',index=False)

# prepara data for cytoscape

In [10]:
import pandas as pd
import numpy as np

virus2host_tax = pd.read_table("/media/chengzw_bk/revision/host_predict/v_2_h_merge_tax_final.txt")
virus2host_tax = virus2host_tax[['class','VC','hic','crisp','blastn']]
# virus2host_tax.hic[virus2host_tax['hic'] == 1] = 1
# virus2host_tax.crisp[virus2host_tax['crisp'] == 1] = 0.1
# virus2host_tax.blastn[virus2host_tax['blastn'] == 1] = 0.01

####存在overlap对应一个order以及多个vc对应一个order
####overlap的情况将vc1/vc2 拆分，分别对应order
####同一个vc对应order对了多次，还是算一次并去重复
# virus2host_tax = virus2host_tax[(~pd.isnull(virus2host_tax.order)) & (~(pd.isnull(virus2host_tax.VC)))]
# virus2host_tax["vc"] = virus2host_tax["VC"].map(lambda x:x.split("/"))
# virus2host_tax = virus2host_tax.explode("vc")[['order','vc']].drop_duplicates()
#virus2host_tax['lable'] = virus2host_tax[['hic','crisp','blastn']].sum(axis=1)
#一共由485个病毒预测到了3785个宿主
#一共预测到5805个virus与host关系
#平均每个病毒预测到12个宿主
#宿主没有分类的有519个
#病毒没有VC的有674个
#既有VC又有order的virus与host关系的4859
virus2host_tax = virus2host_tax[(~pd.isnull(virus2host_tax.VC)) & (~pd.isnull(virus2host_tax['class']))]
virus2host_tax["vc"] = virus2host_tax["VC"].map(lambda x:x.split("/"))
virus2host_tax = virus2host_tax.explode("vc")[['class','vc','blastn','crisp','hic']].reset_index()####就算drop_duplicates()也还存在重复行
virus2host_tax = virus2host_tax.groupby(['class','vc']).agg({'blastn':'max','crisp':'max','hic':'max'}).reset_index()
# 由于存在ovlap的VC和order,overlap的情况将vc1/vc2 拆分，分别对应order,同一个vc对应order对了多次，算一次并去重复,最后得到145个order和VC关系
# hic和crispr共同预测到的5个
# hic和blastn共同预测到的12个
# crispr和blastn共同预测到的8个
virus2host_tax.hic[virus2host_tax['hic'] == 1] = 1
virus2host_tax.crisp[virus2host_tax['crisp'] == 1] = 0.1
virus2host_tax.blastn[virus2host_tax['blastn'] == 1] = 0.01
virus2host_tax['lable'] = virus2host_tax[['hic','crisp','blastn']].sum(axis=1)
virus2host_tax = virus2host_tax[['class','vc','lable']]
virus2host_tax.to_csv("./class2host_relationship.txt",sep='\t',index=False)

/tmp/ipykernel_56553/3163262507.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus2host_tax.hic[virus2host_tax['hic'] == 1] = 1
/tmp/ipykernel_56553/3163262507.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus2host_tax.crisp[virus2host_tax['crisp'] == 1] = 0.1
/tmp/ipykernel_56553/3163262507.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus2host_tax.blastn[virus2host_tax['blastn'] == 1] = 0.01
